In [13]:
# Main file that need help with.
# Input -> Presidio -> Detected Entities -> GPT-4o -> Metric calculations
# Only do WITHIN context filteration (need to extract the sentence where the detected entity belongs to and feed to GPT).
# No need to do out-of-context filteration
import ast
import json
import pandas as pd
from openai import OpenAI
# Load the API key from input/secret.json
# secret_json = read_json_file('input/secret.json')
with open('input/secret.json', 'r', encoding='UTF-8') as file:
    secret_json = json.load(file)
api_key = secret_json['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)

def read_file(filepath: str):
    return pd.read_json(filepath, orient="records")

df = read_file("data/obfuscated_data_06.json")

In [19]:
# # Be aware of the generated cost.

# def check_pii_entities_gpt4o(detected_entities):
#     # Separate entities by category
#     person_entities = [entity for entity in detected_entities if entity[2] == 'PERSON']
#     email_entities = [entity for entity in detected_entities if entity[2] == 'EMAIL_ADDRESS']
#     url_entities = [entity for entity in detected_entities if entity[2] == 'URL']
#     phone_entities = [entity for entity in detected_entities if entity[2] == 'PHONE_NUMBER']

#     # Function to extract the sentence that the PII appears in
#     def extract_sentence(text, s, e):
#         # Find the start of the sentence
#         sentence_start = s
#         while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
#             sentence_start -= 1
        
#         # Find the end of the sentence
#         sentence_end = e
#         while sentence_end < len(text) and text[sentence_end] not in '.!?':
#             sentence_end += 1
        
#         # Extract and return the sentence
#         return text[sentence_start:sentence_end + 1].strip()


#     # Function to call GPT-4o and check for PII
#     def check_entities_with_gpt(entities, category, prompt_template):
#         # Prepare input text for GPT by batching entities
#         prompt = prompt_template + "\n\n"
#         for entity in entities:
#             idx, entity_text, _, positions = entity
#             (s,e) = positions
#             sentence = extract_sentence(df.iloc[idx].full_text, s, e)
            
#             prompt += f"({entity_text}, {sentence})\n"

#         response = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant."},
#                 {"role": "user", "content": prompt}
#             ]
#         )

#         gpt_results = response.choices[0].message.content.strip().split('\n')

#         print(category)
#         print(gpt_results)
#         print("\n")

#         # Process the response and keep entities marked as True (PII detected)
#         filtered_entities = [
#             entity for entity, result in zip(entities, gpt_results) if result.strip().upper() == 'T'
#         ]
        
#         return filtered_entities

#     # Define prompt templates for each category
#     person_prompt = "For each of the following pairs, please decide if the first value is a student's name in the context of the second value (return just a list of 'T's for Trues and 'F's for Falses):"
#     email_prompt = "For each of the following pairs, please decide if the first value is a personal email address in the context of the second value (return just a list of 'T's for Trues and 'F's for Falses):"
#     url_prompt = "For each of the following pairs, please decide if the first value is a personal URL in the context of the second value (return just a list of 'T's for Trues and 'F's for Falses):"
#     phone_prompt = "For each of the following pairs, please decide if the first value is a personal phone number in the context of the second value (return just a list of 'T's for Trues and 'F's for Falses):"

#     # Process each category with GPT-4o
#     valid_person_entities = check_entities_with_gpt(person_entities, 'PERSON', person_prompt)
#     valid_email_entities = check_entities_with_gpt(email_entities, 'EMAIL_ADDRESS', email_prompt)
#     valid_url_entities = check_entities_with_gpt(url_entities, 'URL', url_prompt)
#     valid_phone_entities = check_entities_with_gpt(phone_entities, 'PHONE_NUMBER', phone_prompt)

#     print(valid_url_entities)
#     print("\n")
    
#     # Combine all valid entities into a single list
#     valid_entities = valid_person_entities + valid_email_entities + valid_url_entities + valid_phone_entities
    
#     # TODO: Might need to sort based on file_idx (first item in tuple) before return.
#     return valid_entities


# detected_entities = []
# # with open('output/pii_detected_trf_filtered.txt', 'r') as file:
# #     for line in file:
# #         entity = ast.literal_eval(line.strip())
# #         detected_entities.append(entity)

# with open('output/pii_detected_trf_filtered.txt', 'r') as file:
#         for line in file:
#             entity = ast.literal_eval(line.strip())
#             idx, _, _, _ = entity
#             if idx < 8: # Start from a few examples to try
#                 detected_entities.append(entity)

# # Call the function to filter valid entities
# filtered_entities = check_pii_entities_gpt4o(detected_entities)
# print(f"Filtered entities: {filtered_entities}")

# # Save the filtered out entities in 'output/pii_detected_gpt.txt' or any suitable file name.
# with open('output/pii_detected_gpt.txt', 'w') as f:
#     for entity in filtered_entities:
#         f.write(f"{entity}\n")


PERSON
['T', 'T', 'F', 'F', 'T', 'T']


EMAIL_ADDRESS
['Of course, please provide the pairs for me to evaluate.']


URL
['T', 'T']


PHONE_NUMBER
['Sure, please provide the pairs you would like me to evaluate.']


[(5, 'https://www.santander.com/content/dam/santander-com/es/documentos/informe-anual-de-sostenibilidad/2019/ias-2019-informe-de-', 'URL', (3035, 3158)), (5, 'https://www.greatplacetowork.com/resources/blog/why-is-diversity-inclusion-in-the-workplace-important', 'URL', (4150, 4251))]


Filtered entities: [(0, 'Angela Meyer', 'PERSON', (1039, 1051)), (7, 'Nathalie Sylla', 'PERSON', (52, 66)), (7, 'Nathalie Sylla', 'PERSON', (2281, 2295)), (7, 'Nathalie Sylla', 'PERSON', (3648, 3662)), (5, 'https://www.santander.com/content/dam/santander-com/es/documentos/informe-anual-de-sostenibilidad/2019/ias-2019-informe-de-', 'URL', (3035, 3158)), (5, 'https://www.greatplacetowork.com/resources/blog/why-is-diversity-inclusion-in-the-workplace-important', 'URL', (4150, 4251))]


In [49]:
# Be aware of the generated cost.
# Updated Prompt

# Function to extract the sentence that the PII appears in
def extract_sentence(text, s, e):
    # Find the start of the sentence
    sentence_start = s
    while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
        sentence_start -= 1
    
    # Find the end of the sentence
    sentence_end = e
    while sentence_end < len(text) and text[sentence_end] not in '.!?':
        sentence_end += 1
    
    # Extract and return the sentence
    return text[sentence_start:sentence_end + 1].strip()

# Function to call GPT-4o and check for PII
def check_entities_with_gpt(entities, category, prompt_template):
    # Prepare input text for GPT by batching entities
    prompt = prompt_template + "\n\n"
    for entity in entities:
        idx, entity_text, _, positions = entity
        (s,e) = positions
        sentence = extract_sentence(df.iloc[idx].full_text, s, e)
        print(f"Entity: {entity_text}, Context: {sentence}")
        
        prompt += f"({entity_text}, {sentence})\n"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    gpt_results = response.choices[0].message.content.strip().split('\n')

    print("")
    print(category)
    print("gpt_results:", gpt_results)
    print("\n")

    # Process the response and keep entities marked as True (PII detected)
    filtered_entities = [
        entity for entity, result in zip(entities, gpt_results) if result.strip().upper() == 'T'
    ]
    
    return filtered_entities

def check_pii_entities_gpt4o(detected_entities):
    # Separate entities by category
    person_entities = [entity for entity in detected_entities if entity[2] == 'PERSON']
    email_entities = [entity for entity in detected_entities if entity[2] == 'EMAIL_ADDRESS']
    url_entities = [entity for entity in detected_entities if entity[2] == 'URL']
    phone_entities = [entity for entity in detected_entities if entity[2] == 'PHONE_NUMBER']

    # Define prompt templates for each category
    # person_prompt = "For each of the following pairs, please decide if the first value is a student's name in the context \
    #                 of the second value. Output a list of 'T's for Trues and 'F's for Falses with no additional output; \
    #                 the length of output should equal to the length of input entities, e.g., the output should be a list \
    #                 where each element is a character of either 'T' or 'F'. Do not combine your results to a single string of T and F's."
    person_prompt = "For each of the following pairs, please decide if the first value is a student's name in the context \
                    of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
                    each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
                    Do not combine your results to a single string of T and F's. \
                    Do not wrap your output with additional pairs of \" or brackets."
    email_prompt = "For each of the following pairs, please decide if the first value is a personal email address in the context \
                    of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
                    each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
                    Do not combine your results to a single string of T and F's. \
                    Do not wrap your output with additional pairs of \" or brackets."
    url_prompt = "For each of the following pairs, please decide if the first value is a personal URL in the context \
                    of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
                    each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
                    Do not combine your results to a single string of T and F's. \
                    Do not wrap your output with additional pairs of \" or brackets."
    phone_prompt = "For each of the following pairs, please decide if the first value is a personal phone number in the context \
                    of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
                    each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
                    Do not combine your results to a single string of T and F's. \
                    Do not wrap your output with additional pairs of \" or brackets."

# Define prompt templates for each category
# person_prompt = "For each of the following pairs, please decide if the first value is a student's name in the given context \
#                 of the second value. Output 'T' for True and 'F' for False without additional output. \
#                 Do not wrap your output with additional pairs of \" or brackets. Do not output any thought process."
# email_prompt = "For each of the following pairs, please decide if the first value is a personal email address in the given context \
#                 of the second value. Output 'T' for True and 'F' for False without additional output. \
#                 Do not wrap your output with additional pairs of \" or brackets. Do not output any thought process."
# url_prompt = "For each of the following pairs, please decide if the first value is a personal URL in the given context \
#                 of the second value. Output 'T' for True and 'F' for False without additional output. \
#                 Do not wrap your output with additional pairs of \" or brackets. Do not output any thought process."
# phone_prompt = "For each of the following pairs, please decide if the first value is a personal phone number in the given context \
#                 of the second value. Output 'T' for True and 'F' for False without additional output. \
#                 Do not wrap your output with additional pairs of \" or brackets. Do not output any thought process."

    # Process each category with GPT-4o
    valid_person_entities = check_entities_with_gpt(person_entities, 'PERSON', person_prompt)
    valid_email_entities = check_entities_with_gpt(email_entities, 'EMAIL_ADDRESS', email_prompt)
    valid_url_entities = check_entities_with_gpt(url_entities, 'URL', url_prompt)
    valid_phone_entities = check_entities_with_gpt(phone_entities, 'PHONE_NUMBER', phone_prompt)

    # print(valid_url_entities)
    # print("\n")
    
    # Combine all valid entities into a single list
    valid_entities = valid_person_entities + valid_email_entities + valid_url_entities + valid_phone_entities
    
    # TODO: Might need to sort based on file_idx (first item in tuple) before return.
    return valid_entities


detected_entities = []
# with open('output/pii_detected_trf_filtered.txt', 'r') as file:
#     for line in file:
#         entity = ast.literal_eval(line.strip())
#         detected_entities.append(entity)

with open('output/pii_detected_trf_filtered.txt', 'r') as file:
    for line in file:
        entity = ast.literal_eval(line.strip())
        idx, _, _, _ = entity
        if idx < 21 or idx in [609, 828, 1434, 3518, 3993, 4381, 4777, 4913] or idx in [379, 576, 1440, 1871, 1917, 2063, 2457, 2769]: # Start from a few examples to try
            detected_entities.append(entity)

# Call the function to filter valid entities
filtered_entities = check_pii_entities_gpt4o(detected_entities)
print(f"Filtered entities: {filtered_entities}")

# Save the filtered out entities in 'output/pii_detected_gpt.txt' or any suitable file name.
with open('output/pii_detected_gpt2.txt', 'w') as f:
    for entity in filtered_entities:
        f.write(f"{entity}\n")


Entity: Angela Meyer, Context: Not only  imagine things but visualization is something about able to put our imagination in a way that  others could see our imaginations through our writings or drawings etc… as explained by  Angela Meyer in the session.
Entity: Nathalie Sylla, Context: Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla

Challenge & selection

The tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.
Entity: Buzan T., Context: According to the definition of Buzan T. and Buzan B.
Entity: Buzan B., Context: and Buzan B. (1999, Dessine-moi  l'intelligence.
Entity: Nathalie Sylla, Context: Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla

After modelling the mind map on paper, I propose to the participants a digital visualization of their  work with the addition of color codes, images and interconnections.
Entity: Nathalie Sylla, Context: Design Thinking for innovation reflexion-Avril 20

In [75]:
from typing import List, Tuple
import tiktoken  # Tokenizer library
import ast

# Function to extract the sentence that the PII appears in
def extract_sentence(text, s, e):
    # Find the start of the sentence
    sentence_start = s
    while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
        sentence_start -= 1
    
    # Find the end of the sentence
    sentence_end = e
    while sentence_end < len(text) and text[sentence_end] not in '.!?':
        sentence_end += 1
    
    # Extract and return the sentence
    return text[sentence_start:sentence_end + 1].strip()

# Function to call GPT-4o and check for PII
def check_entities_with_gpt_single(entities, category_str):
    filtered_entities = []

    # Initialize the base conversation history only once
    messages = [{"role": "system", "content": "You are a helpful assistant."}]

    # Process each entity separately within a single conversation context
    for entity in entities:
        idx, entity_text, category, positions = entity
        (s, e) = positions
        sentence = extract_sentence(df.iloc[idx].full_text, s, e)

        # Append the entity and its context to the messages for this specific entity
        entity_message = {"role": "user", "content": f"Is '{entity_text}' a {category_str} in this context: '{sentence}'? Output only 'T' for True or 'F' for False without additional output."}
        messages.append(entity_message)

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

        # Extract the response
        gpt_result = response.choices[0].message.content.strip().upper()

        # Keep only the last response for validation
        if gpt_result == 'T':
            filtered_entities.append(entity)

        # Print for debugging
        print(f"Entity: {entity_text}, Type: {category}, Result: {gpt_result}")

        # Remove the entity message to prevent token accumulation from old inputs
        messages.pop()

    return filtered_entities

def check_pii_entities_gpt4o(detected_entities):
    # Separate entities by category
    person_entities = [entity for entity in detected_entities if entity[2] == 'PERSON']
    email_entities = [entity for entity in detected_entities if entity[2] == 'EMAIL_ADDRESS']
    url_entities = [entity for entity in detected_entities if entity[2] == 'URL']
    phone_entities = [entity for entity in detected_entities if entity[2] == 'PHONE_NUMBER']
    
    category_dict = {
        'PERSON': "student's name",
        'EMAIL_ADDRESS': "personal email address",
        'URL': "personal URL",
        'PHONE_NUMBER': "personal phone number"
    }

    # Process each category with GPT-4o using the single entity approach
    valid_person_entities = check_entities_with_gpt_single(person_entities, category_dict['PERSON'])
    valid_email_entities = check_entities_with_gpt_single(email_entities, category_dict['EMAIL_ADDRESS'])
    valid_url_entities = check_entities_with_gpt_single(url_entities, category_dict['URL'])
    valid_phone_entities = check_entities_with_gpt_single(phone_entities, category_dict['PHONE_NUMBER'])

    # Combine all valid entities into a single list
    valid_entities = valid_person_entities + valid_email_entities + valid_url_entities + valid_phone_entities
    valid_entities_sorted = sorted(valid_entities, key=lambda x: x[0])

    return valid_entities_sorted

# Load entities from file
detected_entities = []
with open('output/pii_detected_trf_filtered.txt', 'r') as file:
    for line in file:
        entity = ast.literal_eval(line.strip())
        # idx, _, _, _ = entity
        # if idx < 21 or idx in [609, 828, 1434, 3518, 3993, 4381, 4777, 4913] or idx in [379, 576, 1440, 1871, 1917, 2063, 2457, 2769]:
        detected_entities.append(entity)

# Call the function to filter valid entities
filtered_entities = check_pii_entities_gpt4o(detected_entities)
print(f"Filtered entities: {filtered_entities}")

# Save the filtered out entities in 'output/pii_detected_gpt.txt' or any suitable file name.
with open('output/pii_detected_gpt4o.txt', 'w') as f:
    for entity in filtered_entities:
        f.write(f"{entity}\n")


Entity: Angela Meyer, Type: PERSON, Result: T
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Buzan T., Type: PERSON, Result: F
Entity: Buzan B., Type: PERSON, Result: F
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Vanesa Chan, Type: PERSON, Result: T
Entity: Aristotle, Type: PERSON, Result: T
Entity: James Cook, Type: PERSON, Result: T
Entity: Challenge, Type: PERSON, Result: F
Entity: Gilberto Gamboa, Type: PERSON, Result: T
Entity: Alessandro, Type: PERSON, Result: T
Entity: Alessandro, Type: PERSON, Result: T
Entity: Sindy Samaca, Type: PERSON, Result: T
Entity: George, Type: PERSON, Result: T
Entity: Geoff, Type: PERSON, Result: F
Entity: Gianni, Type: PERSON, Result: T
Entity: Don Norman, Type: PERSON, Result: F
Entity: Don Norman, Type: PERSON, Result: F
Entity: David Gray’s, Type: PERSON, Result: F
Entity: Chepe Scott, Type: PERSON, Result: T
Entity: Janet Galdamez, Type: PERSON, Result: T
Entity: Personas, Type

In [77]:
from typing import List, Tuple
import tiktoken  # Tokenizer library
import ast

# Function to extract the sentence that the PII appears in
def extract_sentence(text, s, e):
    # Find the start of the sentence
    sentence_start = s
    while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
        sentence_start -= 1
    
    # Find the end of the sentence
    sentence_end = e
    while sentence_end < len(text) and text[sentence_end] not in '.!?':
        sentence_end += 1
    
    # Extract and return the sentence
    return text[sentence_start:sentence_end + 1].strip()

# Function to call GPT-4o and check for PII
def check_entities_with_gpt_single(entities, category_str):
    filtered_entities_with_results = []

    # Initialize the base conversation history only once
    messages = [{"role": "system", "content": "You are a helpful assistant."}]

    # Process each entity separately within a single conversation context
    for entity in entities:
        idx, entity_text, category, positions = entity
        (s, e) = positions
        sentence = extract_sentence(df.iloc[idx].full_text, s, e)

        # Append the entity and its context to the messages for this specific entity
        entity_message = {"role": "user", "content": f"Is '{entity_text}' a {category_str} in this context: '{sentence}'? Output only 'T' for True or 'F' for False without additional output."}
        messages.append(entity_message)

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

        # Extract the response
        gpt_result = response.choices[0].message.content.strip().upper()

        # Append the entity and result to the list
        filtered_entities_with_results.append((idx, entity_text, category, positions, gpt_result))

        # Print for debugging
        print(f"Entity: {entity_text}, Type: {category}, Result: {gpt_result}")

        # Remove the entity message to prevent token accumulation from old inputs
        messages.pop()

    return filtered_entities_with_results

def check_pii_entities_gpt4o(detected_entities):
    # Separate entities by category
    person_entities = [entity for entity in detected_entities if entity[2] == 'PERSON']
    email_entities = [entity for entity in detected_entities if entity[2] == 'EMAIL_ADDRESS']
    url_entities = [entity for entity in detected_entities if entity[2] == 'URL']
    phone_entities = [entity for entity in detected_entities if entity[2] == 'PHONE_NUMBER']
    
    category_dict = {
        'PERSON': "student's name",
        'EMAIL_ADDRESS': "personal email address",
        'URL': "personal URL",
        'PHONE_NUMBER': "personal phone number"
    }

    # Process each category with GPT-4o using the single entity approach
    valid_person_entities = check_entities_with_gpt_single(person_entities, category_dict['PERSON'])
    valid_email_entities = check_entities_with_gpt_single(email_entities, category_dict['EMAIL_ADDRESS'])
    valid_url_entities = check_entities_with_gpt_single(url_entities, category_dict['URL'])
    valid_phone_entities = check_entities_with_gpt_single(phone_entities, category_dict['PHONE_NUMBER'])

    # Combine all valid entities into a single list and sort them by file_idx
    valid_entities_with_results = valid_person_entities + valid_email_entities + valid_url_entities + valid_phone_entities
    valid_entities_sorted = sorted(valid_entities_with_results, key=lambda x: x[0])

    return valid_entities_sorted

# Load entities from file
detected_entities = []
with open('output/pii_detected_trf_filtered.txt', 'r') as file:
    for line in file:
        entity = ast.literal_eval(line.strip())
        idx, _, _, _ = entity
        if idx < 21 or idx in [609, 828, 1434, 3518, 3993, 4381, 4777, 4913] or idx in [379, 576, 1440, 1871, 1917, 2063, 2457, 2769]: # Start from a few examples to try
            detected_entities.append(entity)

# Call the function to filter valid entities
filtered_entities_with_results = check_pii_entities_gpt4o(detected_entities)
print(f"Filtered entities with results: {filtered_entities_with_results}")

# Save the filtered entities with GPT results to a new file
with open('output/pii_detected_gpt4omini_w_result.txt', 'w') as f:
    for entity in filtered_entities_with_results:
        f.write(f"{entity}\n")


Entity: Angela Meyer, Type: PERSON, Result: T
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Buzan T., Type: PERSON, Result: T
Entity: Buzan B., Type: PERSON, Result: T
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Nathalie Sylla, Type: PERSON, Result: T
Entity: Vanesa Chan, Type: PERSON, Result: T
Entity: Aristotle, Type: PERSON, Result: F
Entity: James Cook, Type: PERSON, Result: T
Entity: Challenge, Type: PERSON, Result: F
Entity: Gilberto Gamboa, Type: PERSON, Result: F
Entity: Alessandro, Type: PERSON, Result: T
Entity: Alessandro, Type: PERSON, Result: T
Entity: Sindy Samaca, Type: PERSON, Result: T
Entity: George, Type: PERSON, Result: T
Entity: Geoff, Type: PERSON, Result: T
Entity: Dharmendra Asiri, Type: PERSON, Result: T
Entity: Dharmendra Asiri, Type: PERSON, Result: T
Entity: Abhishek Kumar Brahma, Type: PERSON, Result: T
Entity: Gabriel Bravo, Type: PERSON, Result: T
Entity: Hlengiwe Swetha, Type: PERSON, Result: T
Entity: Tino Swetha, Type: PERSON, Res

In [44]:
# !pip3 install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.7/906.7 kB 14.7 MB/s eta 0:00:00


In [72]:
# # Estimate the number of tokens and the total cost of using GPT-4o to filter detected PII entities.
# import tiktoken  # Tokenizer library
# import ast

# # Function to extract the sentence that the PII appears in
# def extract_sentence(text, s, e):
#     # Find the start of the sentence
#     sentence_start = s
#     while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
#         sentence_start -= 1
    
#     # Find the end of the sentence
#     sentence_end = e
#     while sentence_end < len(text) and text[sentence_end] not in '.!?':
#         sentence_end += 1
    
#     # Extract and return the sentence
#     return text[sentence_start:sentence_end + 1].strip()

# # Function to count tokens in text using GPT-4o tokenizer
# def count_tokens(text, model="gpt-4o"):
#     encoding = tiktoken.encoding_for_model(model)
#     tokens = encoding.encode(text)
#     return len(tokens)

# # Function to estimate the cost of using GPT-4o
# def estimate_cost(file_path, df):
#     total_input_tokens = 0
#     total_output_tokens = 0

#     # Load entities from file
#     with open(file_path, 'r') as file:
#         entities = [ast.literal_eval(line.strip()) for line in file]

#     # Define prompt templates for each category
#     prompts = {
#         "PERSON": "For each of the following pairs, please decide if the first value is a student's name in the context \
#                     of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
#                     each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
#                     Do not combine your results to a single string of T and F's. \
#                     Do not wrap your output with additional pairs of \" or brackets.",
#         "EMAIL_ADDRESS": "For each of the following pairs, please decide if the first value is a personal email address in the context \
#                     of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
#                     each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
#                     Do not combine your results to a single string of T and F's. \
#                     Do not wrap your output with additional pairs of \" or brackets.",
#         "URL": "For each of the following pairs, please decide if the first value is a personal URL in the context \
#                     of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
#                     each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
#                     Do not combine your results to a single string of T and F's. \
#                     Do not wrap your output with additional pairs of \" or brackets.",
#         "PHONE_NUMBER": "For each of the following pairs, please decide if the first value is a personal phone number in the context \
#                     of the second value. Output 'T' for True and 'F' for False without additional output. Output a list where \
#                     each element is either 'T' or 'F', and the length of output should equal to the length of input entities. \
#                     Do not combine your results to a single string of T and F's. \
#                     Do not wrap your output with additional pairs of \" or brackets."
#     }

#     # Count tokens in each prompt template
#     prompt_template_tokens = {category: count_tokens(prompt) for category, prompt in prompts.items()}

#     # Iterate over entities to compute the number of tokens
#     for entity in entities:
#         idx, entity_text, category, positions = entity
#         (s, e) = positions
#         sentence = extract_sentence(df.iloc[idx].full_text, s, e)

#         # Combine entity text and its context for the input
#         input_text = f"({entity_text}, {sentence})"
#         input_tokens = count_tokens(input_text)
        
#         # Add tokens for each entity input and the corresponding prompt
#         total_input_tokens += input_tokens
    
#     total_input_tokens += prompt_template_tokens[category]

#     # Each output 'T' or 'F' corresponds to an input entity
#     total_output_tokens = len(entities)
    
#     # Cost per token in dollars (GPT-4o)
#     input_cost_per_token_gpt4o = 5 / 1000000
#     output_cost_per_token_gpt4o = 15 / 1000000

#     # Cost per token in dollars (GPT-4o-mini)
#     input_cost_per_token_gpt4omini = 0.15 / 1000000
#     output_cost_per_token_gpt4omini = 0.6 / 1000000

#     # Calculate the total cost
#     total_cost_gpt4o = (total_input_tokens * input_cost_per_token_gpt4o) + (total_output_tokens * output_cost_per_token_gpt4o)
#     total_cost_gpt4omini = (total_input_tokens * input_cost_per_token_gpt4omini) + (total_output_tokens * output_cost_per_token_gpt4omini)
    
#     return total_cost_gpt4o, total_cost_gpt4omini, total_input_tokens, total_output_tokens

# # Estimate the cost
# file_path = 'output/pii_detected_trf_filtered.txt'  # Update with the correct path
# cost_4o, cost_4omini, input_tokens, output_tokens = estimate_cost(file_path, df)

# print(f"Total cost (GPT-4o): ${cost_4o:.4f}")
# print(f"Total cost (GPT-4o-mini): ${cost_4omini:.4f}")
# print(f"Total input tokens: {input_tokens}")
# print(f"Total output tokens: {output_tokens}")


Total cost (GPT-4o): $4.1359
Total cost (GPT-4o-mini): $0.1266
Total input tokens: 777158
Total output tokens: 16673


In [76]:
from typing import List, Tuple
import tiktoken  # Tokenizer library
import ast

# Function to extract the sentence that the PII appears in
def extract_sentence(text, s, e):
    # Find the start of the sentence
    sentence_start = s
    while sentence_start > 0 and text[sentence_start - 1] not in '.!?':
        sentence_start -= 1
    
    # Find the end of the sentence
    sentence_end = e
    while sentence_end < len(text) and text[sentence_end] not in '.!?':
        sentence_end += 1
    
    # Extract and return the sentence
    return text[sentence_start:sentence_end + 1].strip()

# Function to count tokens in text using GPT-4o tokenizer
def count_tokens(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# Function to estimate the cost of using GPT-4o
def estimate_cost(file_path, df):
    total_input_tokens = 0
    total_output_tokens = 0

    # Load entities from file
    with open(file_path, 'r') as file:
        entities = [ast.literal_eval(line.strip()) for line in file]

    # Define category strings for each type of entity
    category_dict = {
        'PERSON': "student's name",
        'EMAIL_ADDRESS': "personal email address",
        'URL': "personal URL",
        'PHONE_NUMBER': "personal phone number"
    }

    # Iterate over entities to compute the number of tokens
    for entity in entities:
        idx, entity_text, category, positions = entity
        (s, e) = positions
        sentence = extract_sentence(df.iloc[idx].full_text, s, e)

        # Create the specific prompt for each entity
        prompt = f"Is '{entity_text}' a {category_dict[category]} in this context: '{sentence}'? Output only 'T' for True or 'F' for False without additional output."
        
        # Calculate the number of tokens in the prompt and input
        input_tokens = count_tokens(prompt)
        total_input_tokens += input_tokens

    # Each output 'T' or 'F' corresponds to an input entity
    total_output_tokens = len(entities)
    
    # Cost per token in dollars (GPT-4o)
    input_cost_per_token_gpt4o = 5 / 1000000
    output_cost_per_token_gpt4o = 15 / 1000000

    # Cost per token in dollars (GPT-4o-mini)
    input_cost_per_token_gpt4omini = 0.15 / 1000000
    output_cost_per_token_gpt4omini = 0.6 / 1000000

    # Calculate the total cost
    total_cost_gpt4o = (total_input_tokens * input_cost_per_token_gpt4o) + (total_output_tokens * output_cost_per_token_gpt4o)
    total_cost_gpt4omini = (total_input_tokens * input_cost_per_token_gpt4omini) + (total_output_tokens * output_cost_per_token_gpt4omini)
    
    return total_cost_gpt4o, total_cost_gpt4omini, total_input_tokens, total_output_tokens

# Estimate the cost
file_path = 'output/pii_detected_trf_filtered.txt'  # Update with the correct path
cost_4o, cost_4omini, input_tokens, output_tokens = estimate_cost(file_path, df)

print(f"Total cost (GPT-4o): ${cost_4o:.4f}")
print(f"Total cost (GPT-4o-mini): ${cost_4omini:.4f}")
print(f"Total input tokens: {input_tokens}")
print(f"Total output tokens: {output_tokens}")


Total cost (GPT-4o): $6.4005
Total cost (GPT-4o-mini): $0.1945
Total input tokens: 1230090
Total output tokens: 16673


In [11]:
# Example of loading a text file
import pandas as pd

def read_file(filepath: str):
    return pd.read_json(filepath, orient="records")

df = read_file("data/obfuscated_data_06.json")
print(df.shape)
print(list(df.columns))

(22688, 5)
['full_text', 'document', 'tokens', 'trailing_whitespace', 'labels']


In [19]:
# Example of loading a text file (Continued)
file_idx = 5
input_text = df.iloc[file_idx].full_text
print(input_text)

Diversity and inclusion – Storytelling

Challenge     I am a lawyer, trying to motivate diversity and inclusion into the company where I work.  According to a study of Santander bank promoting inclusion at all levels attracts, develops  and retains the best talent. I am going to explain the meanings of diversity and inclusion to  demonstrate a background of what I’m trying to accomplish. Diversity takes many forms,  gender or race, for example are visible, while people’s values, religion or educational  backgrounds are not. Inclusion is a sense of belonging and makes people feel respected  and valued.1    Some workers have fewer opportunities because they are women, and others with a  disability don’t have equal access to work with us. They need to be empowered and  supported. I use an old story between a man and a woman that have the same position  but different salaries to show and  convince you to make changes.    Selection    We have been trying different tools, like mind mapping, 

In [ ]:
# If you can record the cost of provide a reasonable estimate of the cost generated from GPT, that'll be great.

In [ ]:
# When you finish, please go to evaluator.ipynb and run  calculate_metrics('output/pii_detected_gpt.txt')
# and go to evaluator_categories.ipynb and run  evaluate_detected_entities('output/pii_detected_gpt.txt')
# The two lines of code are marked with TODO.